In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/semi-test/semi_test/e802159c.jpg
/kaggle/input/semi-test/semi_test/c6581110.jpg
/kaggle/input/semi-test/semi_test/8cc1b796.jpg
/kaggle/input/semi-test/semi_test/2a6bf2e2.jpg
/kaggle/input/semi-test/semi_test/a518d741.jpg
/kaggle/input/semi-test/semi_test/3d088a6a.jpg
/kaggle/input/semi-test/semi_test/70dd6aa2.jpg
/kaggle/input/semi-test/semi_test/a1c9d6c7.jpg
/kaggle/input/semi-test/semi_test/8c4f7121.jpg
/kaggle/input/semi-test/semi_test/33b2ffde.jpg
/kaggle/input/semi-test/semi_test/af4e1ebe.jpg
/kaggle/input/semi-test/semi_test/7e24dbaa.jpg
/kaggle/input/semi-test/semi_test/6bf2c438.jpg
/kaggle/input/semi-test/semi_test/3565ecef.jpg
/kaggle/input/semi-test/semi_test/554d5db0.jpg
/kaggle/input/semi-test/semi_test/bc2f69ad.jpg
/kaggle/input/semi-test/semi_test/59e6c6ec.jpg
/kaggle/input/semi-test/semi_test/807d2f54.jpg
/kaggle/input/semi-test/semi_test/8846edbf.jpg
/kaggle/input/semi-test/semi_test/962ceb05.jpg
/kaggle/input/semi-test/semi_test/70b80683.jpg
/kaggle/input

In [93]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import pandas as pd
from PIL import Image
import torchvision
import pandas as pd
from torch.utils.data import Dataset


# # Define a custom dataset class for loading the image pairs and labels
# class WriterDataset(Dataset):
#     def __init__(self, csv_file, root_dir, transform=None):
#         self.data = pd.read_csv(csv_file)
#         self.root_dir = root_dir
#         self.transform = transform

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         img1_path = os.path.join(self.root_dir, self.data.iloc[idx, 0])
#         img2_path = os.path.join(self.root_dir, self.data.iloc[idx, 1])
#         label = self.data.iloc[idx, 2]

#         img1 = Image.open(img1_path).convert('L')  # Convert to grayscale
#         img2 = Image.open(img2_path).convert('L')

#         if self.transform:
#             img1 = self.transform(img1)
#             img2 = self.transform(img2)

#         return img1, img2, label


class WriterDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img1_path = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        img2_path = os.path.join(self.root_dir, self.data.iloc[idx, 1])
        label = self.data.iloc[idx, 2]  # Assuming the label column is at index 2

        img1 = Image.open(img1_path).convert('L')
        img2 = Image.open(img2_path).convert('L')

        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2, label



# Define the network architecture
class WriterClassifier(nn.Module):
    def __init__(self):
        super(WriterClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 250 * 25, 128)
        self.fc2 = nn.Linear(128, 2)  # Output size 2 for binary classification

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


# Inside the WriterDataset class
def __getitem__(self, idx):
    img1_path = os.path.join(self.root_dir, self.data.iloc[idx, 0])
    img2_path = os.path.join(self.root_dir, self.data.iloc[idx, 1])
    label = self.data.iloc[idx, 2]

    img1 = Image.open(img1_path).convert('RGB')
    img2 = Image.open(img2_path).convert('RGB')

    if self.transform:
        img1 = self.transform(img1)
        img2 = self.transform(img2)

    return img1, img2, label

# Define the training function
def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()

    for epoch in range(num_epochs):
        running_loss = 0.0

        for inputs1, inputs2, labels in train_loader:
            inputs1 = inputs1.to(device)
            inputs2 = inputs2.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs1 - inputs2)  # Input difference
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")


        
        
# Set random seeds for reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Set device (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((50, 500)),
    transforms.ToTensor()
])


In [70]:
# Define paths to train and val csv files and image folders
train_csv_file = '/kaggle/input/combined-file/combined_file.csv'
val_csv_file = '/kaggle/input/dataset/dataset/val.csv'
train_root_dir = '/kaggle/input/dataset/dataset/train'  # Replace with the actual path to your train image folder
val_root_dir = '/kaggle/input/dataset/dataset/val'  # Replace with the actual path to your val image folder

In [71]:
val_dataset = WriterDataset(val_csv_file, val_root_dir, transform=transform)

In [72]:

# Create train and val datasets
train_dataset = WriterDataset(train_csv_file, train_root_dir, transform=transform)

In [73]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Create an instance of the model
model = WriterClassifier().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



# Train the model
num_epochs = 7
train(model, train_loader, criterion, optimizer, num_epochs)

# Evaluate the model on the validation set
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for inputs1, inputs2, labels in val_loader:
        inputs1 = inputs1.to(device)
        inputs2 = inputs2.to(device)
        labels = labels.to(device)

        outputs = model(inputs1 - inputs2)  # Input difference
        _, predicted = torch.max(outputs.data, 1)
        
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
auc = roc_auc_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

print(f"Validation Accuracy: {accuracy:.2f}%")
print(f"AUC: {auc:.4f}")
print(f"F1 Score: {f1:.4f}")


Epoch 1/7, Loss: 0.7147
Epoch 2/7, Loss: 0.5711
Epoch 3/7, Loss: 0.5522
Epoch 4/7, Loss: 0.5347
Epoch 5/7, Loss: 0.5170
Epoch 6/7, Loss: 0.4998
Epoch 7/7, Loss: 0.4694
Validation Accuracy: 0.66%
AUC: 0.6558
F1 Score: 0.5796


In [76]:
import os

for dirname, _, filenames in os.walk('/kaggle/working/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/working/__notebook_source__.ipynb


In [83]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd

# Define a custom dataset class for loading the image pairs and labels
class WriterTestDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img1_path = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        img2_path = os.path.join(self.root_dir, self.data.iloc[idx, 1])

        img1 = Image.open(img1_path).convert('L')  # Convert to grayscale
        img2 = Image.open(img2_path).convert('L')

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2

# Define paths to test csv file and image folder
test_csv_file = '/kaggle/input/test-csv/test (2).csv'  # Replace with the actual path to your test.csv file
test_root_dir = '/kaggle/input/semi-test/semi_test'  # Replace with the actual path to your test image folder

# Create test dataset
test_dataset = WriterTestDataset(test_csv_file, test_root_dir, transform=transform)

# Create test data loader
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Generate predictions for the test set
model.eval()
predictions = []

with torch.no_grad():
    for inputs1, inputs2 in test_loader:
        inputs1 = inputs1.to(device)
        inputs2 = inputs2.to(device)

        outputs = model(inputs1 - inputs2)  # Input difference
        probabilities = nn.Softmax(dim=1)(outputs)
        predictions.extend(probabilities[:, 1].cpu().numpy())  # Select the probability for class 1 (same writer)

# Create submission dataframe
submission_df = pd.read_csv(test_csv_file)
submission_df['id'] = submission_df['img1_name'] + '_' + submission_df['img2_name']
submission_df['proba'] = predictions

# Save submission file
submission_file = 'submission.csv'
submission_df[['id', 'proba']].to_csv(submission_file, index=False)

In [85]:
import pandas as pd

# Read the submission file
submission_file = 'submission.csv'  # Replace with the actual path to your submission.csv file
submission_df = pd.read_csv(submission_file)

# Print the contents of the submission file
print(submission_df)


                              id     proba
0      3861e092.jpg_b642bddc.jpg  0.711703
1      776e0055.jpg_2d4e7d20.jpg  0.428957
2      4bb7f909.jpg_af1b0fbf.jpg  0.428957
3      0abee1d8.jpg_1c2641c9.jpg  0.877310
4      1e18f51e.jpg_9fcf5c47.jpg  0.014230
...                          ...       ...
10581  c958571b.jpg_2df4a5a6.jpg  0.833761
10582  6493f6be.jpg_33faf06a.jpg  0.428957
10583  4dc5259f.jpg_0e85870e.jpg  0.933035
10584  8771a705.jpg_e41f0ede.jpg  0.958152
10585  84715fe3.jpg_04827f85.jpg  0.005710

[10586 rows x 2 columns]
